In [2]:
from datasets import load_dataset
tldr_dataset = load_dataset("CarperAI/openai_summarize_tldr")
print(tldr_dataset['train'][0])

/Users/helenz/Desktop/mlx-summariser/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'prompt': "SUBREDDIT: r/relationships\nTITLE: I (f/22) have to figure out if I want to still know these girls or not and would hate to sound insulting\nPOST: Not sure if this belongs here but it's worth a try. \n\nBackstory:\nWhen I (f/22) went through my first real breakup 2 years ago because he needed space after a year of dating roand  it effected me more than I thought. It was a horrible time in my life due to living with my mother and finally having the chance to cut her out of my life. I can admit because of it was an emotional wreck and this guy was stable and didn't know how to deal with me. We ended by him avoiding for a month or so after going to a festival with my friends. When I think back I wish he just ended. So after he ended it added my depression I suffered but my friends helped me through it and I got rid of everything from him along with cutting contact. \n\nNow: Its been almost 3 years now and I've gotten better after counselling and mild anti depressants. My mothe

In [ ]:
print(tldr_dataset)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'label'],
        num_rows: 116722
    })
    test: Dataset({
        features: ['prompt', 'label'],
        num_rows: 6553
    })
    valid: Dataset({
        features: ['prompt', 'label'],
        num_rows: 6447
    })
})


In [5]:
comparison_dataset = load_dataset("CarperAI/openai_summarize_comparisons")
print(comparison_dataset['train'][0])

Generating valid2 split: 100%|██████████| 50715/50715 [00:00<00:00, 593207.51 examples/s]


{'prompt': 'SUBREDDIT: r/relationships\nTITLE: To admit or not to admit snooping...\nPOST: I [25M] have snooped in the past and copped up to it to my gf [25F] of 6 years.  We talked it through.  It had been a year or two since the last time.  That\'s an issue I\'m working on.\n\nNow she has a new close male work friend.  I won\'t go into details, but she hides things from me with him and does other things to make me a bit suspicious.  So...I snooped again, and this time, all texts from her new friend have been deleted and I saw a google search for "how to get over a guy" near some searches of his name and views of his Facebook profile.\n\nI asked her about this guy, not mentioning the snooping, and she denied any feelings, we talked for a long time about our relationship and she insisted that she only loves me and I mean the world to her, and that she really wants to work towards getting this relationship back out of the rut we\'ve been in (we both work all the time and barely see each

In [6]:
print(comparison_dataset)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 92534
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 83629
    })
    valid1: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 33082
    })
    valid2: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 50715
    })
})


In [ ]:
from datasets import load_dataset
from torch.utils.data import Dataset
import torch

class TLDRDataset(Dataset):
    def __init__(self, train_path, tokenizer, split, max_length=550):
        dataset = load_dataset(train_path, split=split)
        self.examples = [sample["prompt"] + sample["label"] for sample in dataset]
        self.examples = self.examples[:2000] if "valid" in split else self.examples
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            self.examples[idx], truncation=True, max_length=self.max_length, padding="max_length"
        )
        return {
            "input_ids": torch.tensor(enc["input_ids"]),
            "attention_mask": torch.tensor(enc["attention_mask"]),
            "labels": torch.tensor(enc["input_ids"]),  # teacher forcing
        }

In [4]:
tldr_dataset_test = tldr_dataset["train"].select(range(100))
examples = [sample["prompt"] + sample["label"] for sample in tldr_dataset_test]
print(examples[0])

SUBREDDIT: r/relationships
TITLE: I (f/22) have to figure out if I want to still know these girls or not and would hate to sound insulting
POST: Not sure if this belongs here but it's worth a try. 

Backstory:
When I (f/22) went through my first real breakup 2 years ago because he needed space after a year of dating roand  it effected me more than I thought. It was a horrible time in my life due to living with my mother and finally having the chance to cut her out of my life. I can admit because of it was an emotional wreck and this guy was stable and didn't know how to deal with me. We ended by him avoiding for a month or so after going to a festival with my friends. When I think back I wish he just ended. So after he ended it added my depression I suffered but my friends helped me through it and I got rid of everything from him along with cutting contact. 

Now: Its been almost 3 years now and I've gotten better after counselling and mild anti depressants. My mother has been out of m

In [ ]:
from transformers import AutoTokenizer
qwen_name = "Qwen/Qwen3-0.6B-Base"
qwen_tokenizer = AutoTokenizer.from_pretrained(qwen_name, trust_remote_code=True)


In [8]:
qwen_tokenizer.pad_token = qwen_tokenizer.eos_token
print(qwen_tokenizer.eos_token)

<|endoftext|>


In [10]:
index = 0
enc = qwen_tokenizer(
    examples[index], truncation=True, max_length=550, padding="max_length"
)
print(enc["input_ids"])

[29038, 787, 4103, 952, 25, 435, 14, 85824, 198, 50328, 25, 358, 320, 69, 14, 17, 17, 8, 614, 311, 7071, 700, 421, 358, 1366, 311, 2058, 1414, 1493, 7571, 476, 537, 323, 1035, 12213, 311, 5112, 67092, 198, 2946, 25, 2806, 2704, 421, 419, 17180, 1588, 714, 432, 594, 5802, 264, 1430, 13, 4710, 3707, 26485, 510, 4498, 358, 320, 69, 14, 17, 17, 8, 3937, 1526, 847, 1156, 1931, 84498, 220, 17, 1635, 4134, 1576, 566, 4362, 3550, 1283, 264, 1042, 315, 4924, 926, 437, 220, 432, 88489, 752, 803, 1091, 358, 3381, 13, 1084, 572, 264, 27102, 882, 304, 847, 2272, 4152, 311, 5382, 448, 847, 6554, 323, 5499, 3432, 279, 6012, 311, 3931, 1059, 700, 315, 847, 2272, 13, 358, 646, 16698, 1576, 315, 432, 572, 458, 14269, 35750, 323, 419, 7412, 572, 15175, 323, 3207, 944, 1414, 1246, 311, 3484, 448, 752, 13, 1205, 9482, 553, 1435, 30426, 369, 264, 2254, 476, 773, 1283, 2087, 311, 264, 18780, 448, 847, 4780, 13, 3197, 358, 1744, 1182, 358, 6426, 566, 1101, 9482, 13, 2055, 1283, 566, 9482, 432, 3694, 847, 1821

In [11]:
print(enc["attention_mask"])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(qwen_name, use_cache=False)
# Resize token embeddings / adjust token count
model.resize_token_embeddings(len(qwen_tokenizer)) 
model.config.pad_token_id = qwen_tokenizer.eos_token_id

In [13]:
print(model)

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151669, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [14]:
print(f"\n🔠 Tokenizer vocab size: {len(qwen_tokenizer)}")


🔠 Tokenizer vocab size: 151669
